# Parallelizing Image Convolution

## Learning Goals

By the end of this tutorial, you will be able to:

- Employ three parallelization libraries to speed up a serial process.
- Calculate the speedup of the different approaches shown.
- Evaluate which library is suited to your task.

## Introduction

This notebook shows how to speed up an image convolution task using these three libraries:

* Ray: an open-source unified compute framework that makes it easy to scale AI and Python workloads.
* Multiprocessing: part of the standard library; supports spawning processes using an API similar to the threading module; offers both local and remote concurrency, effectively side-stepping the Global Interpreter Lock by using subprocesses instead of threads.
* Dask: developed to natively scale computational packages like numpy, pandas and scikit-learn, and the surrounding ecosystem, to multi-core machines and distributed clusters when datasets exceed memory.

## Imports

* _multiprocessing.Pool_ for multiprocessing using the standard library
* _time_ for timing the processes
* _dask.distributed.Client_ for making a local Dask cluster
* _numpy_ and _scipy.signal_ for numerical work
* _psutil_ for finding the available processors on your machine
* _ray_ for scaling up Python tasks

In [1]:
from multiprocessing import Pool
import time

from dask.distributed import Client
import numpy as np
import psutil
import scipy.signal
import ray

2024-08-20 06:22:44,557	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Find the cpus available

Find and print the number of cpus
(taken from https://towardsdatascience.com/10x-faster-parallel-python-without-python-multiprocessing-e5017c93cce1)

In [2]:
num_cpus = psutil.cpu_count(logical=True)
print(num_cpus)

4


## Process serially using a conventional loop

Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result.

In [3]:
def fconv(image, random_filter):
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

In [4]:
filters = [np.random.normal(size=(4, 4)) for _ in range(num_cpus)]

Process 100 iterations serially, then extrapolate to num_cpus*100

In [5]:
start = time.time()
num_iter = 100
image = np.zeros((3000, 3000))
for i in range(num_iter):
    result = fconv(image, filters[i % num_cpus])
duration_conv = time.time() - start
print("(scaled) conventional duration for {:d} iterations = {:.1f} seconds"
      .format(num_cpus*num_iter, duration_conv*num_cpus))

(scaled) conventional duration for 400 iterations = 136.3 seconds


## Process in parallel using Ray

[Documentation for ray](https://docs.ray.io/en/latest/)

The warning raised by `ray.init` only affects shared object usage, which is not an issue for this tutorial. It may harm performance in other scenarios.

In [6]:
ray.init(num_cpus=num_cpus)

2024-08-20 06:23:20,609	INFO worker.py:1781 -- Started a local Ray instance.


Python version:,3.11.9
Ray version:,2.34.0


Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result. To use Ray, we decorate the function that is doing the work.

In [7]:
@ray.remote
def fray(image, random_filter):
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

In the following loop, `ray.put` places the image into shared memory. The call to `ray.get` retrieves the result.

In [8]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    image_id = ray.put(image)
    ray.get([fray.remote(image_id, filters[i]) for i in range(num_cpus)])
duration_ray = time.time() - start
print("Ray duration = {:.1f}, speedup = {:.2f}"
      .format(duration_ray, duration_conv*num_cpus / duration_ray))

Ray duration = 74.6, speedup = 1.83


In [9]:
ray.shutdown()

## Process in parallel using multiprocessing

[Documentation for multiprocessing](https://docs.python.org/3/library/multiprocessing.html)

Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result. The call to the function has a slightly different form than that for the serial loop.

In [10]:
def fmp(args):
    image, random_filter = args
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

Use a multiprocessing pool with the number of cpus we found earlier.

In [11]:
pool = Pool(num_cpus)

Using `pool.map` is the closest analog in multiprocessing to the Ray API.

In [12]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    pool.map(fmp, zip(num_cpus * [image], filters))
duration_mp = time.time() - start
print("Multiprocessing duration = {:.1f}, speedup = {:.2f}"
      .format(duration_mp, duration_conv*num_cpus / duration_mp))

Multiprocessing duration = 140.2, speedup = 0.97


## Process using Dask

[Documentation for Dask](https://www.dask.org/get-started)

Define a Dask distributed client with number of workers set to the number of cpus we found earlier, and with one thread per worker.

In [13]:
client = Client(n_workers=num_cpus, threads_per_worker=1)

In [14]:
print(client)

<Client: 'tcp://127.0.0.1:33625' processes=4 threads=4, memory=15.61 GiB>


Dask recommends scattering the large inputs across the workers, though this makes little difference in execution time.

In [15]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    for j in range(num_cpus):
        big_future = client.scatter((image, filters[j % num_cpus]))
        future = client.submit(fconv, big_future)
duration_dask = time.time() - start
print("Dask duration = {:.1f}, speedup = {:.2f}"
      .format(duration_dask, duration_conv*num_cpus / duration_dask))

2024-08-20 06:26:59,687 - distributed.worker - ERROR - Compute Failed
Key:       fconv-62d3fb08cdcd4722add20d9c193727a4
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.04511129, -0.36965459, -0.95263628, -0.14601954],
       [-0.59376302, -0.84903493, -0.89073568, -0.66741073],
       [-0.47099116, -1.10903786,  1.44708216,  0.12989074],
       [-1.48281771, -1.25057209, -0.32882889,  0.46543018]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-20 06:26:59,792 - distributed.worker - ERROR - Compute Failed
Key:       fconv-1ac3fce8ca6810762f6ba41bdd63fbbd
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-20 06:26:59,966 - distributed.worker - ERROR - Compute Failed
Key:       fconv-9b7b36da0833e49e528167886f9252af
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 2.46791787, -0.82683262,  1.8737225 ,  1.62838958],
       [ 1.47982596,  0.31444343, -0.44400358,  1.90457769],
       [-1.37967439,  1.53892987, -0.91005709,  0.1812818 ],
       [-0.63523779, -1.30489074, -0.1415764 , -1.05707001]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-20 06:27:00,055 - distributed.worker - ERROR - Compute Failed
Key:       fconv-ae5f88f9823e61e9b58c09b64e526d68
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-20 06:27:00,253 - distributed.worker - ERROR - Compute Failed
Key:       fconv-ac22d9c2e5cfd448a01b8ef294a4f133
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.90332109,  0.39267784,  0.64363399, -0.87981171],
       [ 1.55428077,  0.42831882, -1.26057054, -0.83241402],
       [-0.18448306,  0.4678934 ,  0.96580758, -0.66310292],
       [-0.40874477, -0.61804064, -1.72640124,  0.25839018]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-20 06:27:00,342 - distributed.worker - ERROR - Compute Failed
Key:       fconv-443bc3a586bb1b4a93b6a3ef66393997
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-20 06:27:00,525 - distributed.worker - ERROR - Compute Failed
Key:       fconv-908c517e50cbbd8645cb7ea9966d5514
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.83881391, -0.79365604, -1.33425365, -0.18892292],
       [ 1.4698928 ,  1.14847572,  0.27104348, -1.5250652 ],
       [ 2.91514976, -0.65007908, -0.53265913,  0.41531753],
       [-0.01336496, -1.53023586,  0.30497577, -2.23417473]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-20 06:27:00,616 - distributed.worker - ERROR - Compute Failed
Key:       fconv-efdee41206bc8dbd74eab69aad2ef928
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-20 06:27:00,794 - distributed.worker - ERROR - Compute Failed
Key:       fconv-824ed747ef772d360ee1ddee74436097
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.04511129, -0.36965459, -0.95263628, -0.14601954],
       [-0.59376302, -0.84903493, -0.89073568, -0.66741073],
       [-0.47099116, -1.10903786,  1.44708216,  0.12989074],
       [-1.48281771, -1.25057209, -0.32882889,  0.46543018]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-20 06:27:00,887 - distributed.worker - ERROR - Compute Failed
Key:       fconv-0d0c260c5f2a6d78bc00fa07a62b8150
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

...

2024-08-20 06:27:35,200 - distributed.worker - ERROR - Compute Failed
Key:       fconv-574fbcfa555e83f52a55cd02bf931a8d
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 2.46791787, -0.82683262,  1.8737225 ,  1.62838958],
       [ 1.47982596,  0.31444343, -0.44400358,  1.90457769],
       [-1.37967439,  1.53892987, -0.91005709,  0.1812818 ],
       [-0.63523779, -1.30489074, -0.1415764 , -1.05707001]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-20 06:27:35,287 - distributed.worker - ERROR - Compute Failed
Key:       fconv-4164add2d7cf6b5295e88bf3cc4e64b7
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-20 06:27:35,465 - distributed.worker - ERROR - Compute Failed
Key:       fconv-dbfa71c5ec45472827a2a20e690a335d
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.90332109,  0.39267784,  0.64363399, -0.87981171],
       [ 1.55428077,  0.42831882, -1.26057054, -0.83241402],
       [-0.18448306,  0.4678934 ,  0.96580758, -0.66310292],
       [-0.40874477, -0.61804064, -1.72640124,  0.25839018]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-20 06:27:35,554 - distributed.worker - ERROR - Compute Failed
Key:       fconv-88466ece57362ca4e185a1d01c1d49c5
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-20 06:27:35,753 - distributed.worker - ERROR - Compute Failed
Key:       fconv-5e5c894a53913a7e30ff61d68f2f4247
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.83881391, -0.79365604, -1.33425365, -0.18892292],
       [ 1.4698928 ,  1.14847572,  0.27104348, -1.5250652 ],
       [ 2.91514976, -0.65007908, -0.53265913,  0.41531753],
       [-0.01336496, -1.53023586,  0.30497577, -2.23417473]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-20 06:27:35,844 - distributed.worker - ERROR - Compute Failed
Key:       fconv-4f0fbf7eae6a59d53ba956f30fe75d32
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

Dask duration = 36.9, speedup = 3.69


2024-08-20 06:27:36,017 - distributed.worker - ERROR - Compute Failed
Key:       fconv-16ab997d9644c2078aea39c76691be47
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.04511129, -0.36965459, -0.95263628, -0.14601954],
       [-0.59376302, -0.84903493, -0.89073568, -0.66741073],
       [-0.47099116, -1.10903786,  1.44708216,  0.12989074],
       [-1.48281771, -1.25057209, -0.32882889,  0.46543018]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''



In [16]:
client.close()

## Conclusions

* Ray is the most effective at speeding up the convolution workload by fully utilizing all available processes
* Multiprocessing is second in effectiveness
* Dask delivers the least speedup; perhaps due to having only six processes on the dask.distributed client

## About this notebook

This notebook was developed by David Shupe (shupe@ipac.caltech.edu) in conjunction with Jessica Krick and the IRSA Science Platform team at IPAC.

## Citations

If you use these software packages in your work, please use the following citations:

* Dask: Dask Development Team (2016). Dask: Library for dynamic task scheduling. URL https://dask.org
* Ray: The Ray Development Team. URL https://docs.ray.io